In [1]:
import pandas as pd
import numpy as np
import gdown 
import scipy.sparse as sps
import pickle
from tqdm import tqdm
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

## Загрузка данных

In [21]:
# Загрузка тестовых данных
gdown.download(url="https://drive.google.com/uc?export=download&id=1Ud6jFto6e7FW5y0LxxwX8afpmTrvA0u-",  
               output="test_transacrion_df.csv",
               quiet=False)
# Загрузка датафрейма с тестовыми данными
test_df = pd.read_csv("test_transacrion_df.csv", index_col=0)
test_all_df = test_df.copy()

Downloading...
From: https://drive.google.com/uc?export=download&id=1Ud6jFto6e7FW5y0LxxwX8afpmTrvA0u-
To: /home/noname/projects/hse_mlds_recsys_project/ML/test_transacrion_df.csv
100%|██████████| 52.4M/52.4M [00:08<00:00, 6.36MB/s]


In [22]:
# Загрузка кодировщика user_id
gdown.download(url="https://drive.google.com/uc?export=download&id=1eodl9OlaYy3NTu9TpbtPLHNurXxeHlhh",  
               output="encoder.pkl",
               quiet=False)
# Загрузка кодировщика
with open("encoder.pkl", "rb") as f:
    encoder = pickle.load(f)

Downloading...
From: https://drive.google.com/uc?export=download&id=1eodl9OlaYy3NTu9TpbtPLHNurXxeHlhh
To: /home/noname/projects/hse_mlds_recsys_project/ML/encoder.pkl
100%|██████████| 800k/800k [00:00<00:00, 3.78MB/s]


In [23]:
# Загрузка датафрейма с рейтингами
gdown.download(url="https://drive.google.com/uc?export=download&id=1epGrpzB8BEC2t5Od3hrL3x07B1VZIm3c",  
               output="ratings.csv",
               quiet=False)
rating_df = pd.read_csv("ratings.csv", index_col=0)

Downloading...
From: https://drive.google.com/uc?export=download&id=1epGrpzB8BEC2t5Od3hrL3x07B1VZIm3c
To: /home/noname/projects/hse_mlds_recsys_project/ML/ratings.csv
100%|██████████| 211M/211M [00:18<00:00, 11.3MB/s] 


In [24]:
# Перекодировка пользователей
init_test_df = test_df.copy()
test_df['user_id'] = encoder.transform(test_df['user_id'])
test_df = test_df[['user_id', 'product_id']].drop_duplicates()
test_df['rating'] = 1
rating_df['user_id'] = encoder.transform(rating_df['user_id'])

In [25]:
# Приведение типов к менее тяжеловесным
rating_df["user_id"] = pd.to_numeric(rating_df["user_id"], downcast="integer")
rating_df["product_id"] = pd.to_numeric(rating_df["product_id"], downcast="integer")
rating_df["rating"] = pd.to_numeric(rating_df["rating"], downcast="integer")
test_df["user_id"] = pd.to_numeric(test_df["user_id"], downcast="integer")
test_df["product_id"] = pd.to_numeric(test_df["product_id"], downcast="integer")
test_df["rating"] = pd.to_numeric(test_df["rating"], downcast="integer")

In [26]:
def make_sparse(dataset: pd.DataFrame) -> sps.coo_matrix:
    row = dataset["user_id"].to_numpy()
    col = dataset["product_id"].to_numpy()
    data = dataset["rating"].to_numpy()
    return sps.coo_matrix((data, (row, col)))

In [27]:
train_sparse = make_sparse(dataset=rating_df)
test_sparse = make_sparse(dataset=test_df)

## Метрика качества

In [30]:
def precision_at_k(relevant, predicted, k: int = 10):
    """ 
        Функиця расчета Precision@k
        relevant - релевантные items для одного пользователя
        predicted - рекомендованные items для одного пользователя
    """
    return len(set(relevant[:k]) & set(predicted[:k]))/k 

def rel_item(relevant, predicted):
    """
        Функция рассчитывает количество релевантных Item
    """
    result = [0]*max(len(relevant), len(predicted))
    items = min(len(relevant), len(predicted))
    for i in range(items):
        result[i] = int(relevant[i] == predicted[i])
    return result  


def ap_at_k(relevant, predicted, k: int = 10):
    """ 
        Функция расчета AP@k
        relevant - релевантные items для одного пользователя
        predicted - рекомендованные items для одного пользователя
    """
    y_i = rel_item(relevant=relevant, predicted=predicted)
    p_at_i = [0]*k
    iter_cnt = min(len(relevant), k)
    for i in range(1, iter_cnt+1):
        p_at_i[i-1] = precision_at_k(relevant=relevant, predicted=predicted, k=i)
    return sum([y*p/k for y, p in zip(y_i, p_at_i)])

def map_at_k(relevant, predicted, k: int = 10):
    """ 
        Функция расчета MAP@k
        relevant список по всем пользователям с их релевантными items
        predicted список по всем пользователям с их рекомендованными items
    """
    users = len(relevant)
    sum_apk = 0
    for user in range(users):
        sum_apk += ap_at_k(relevant=relevant[user], predicted=predicted[user], k=k)
    return sum_apk/users


def nap_at_k(relevant, predicted, k: int = 10):
    """ 
        Функция расчета normalize AP@k
        relevant - релевантные items для одного пользователя
        predicted - рекомендованные items для одного пользователя
    """
    y_i = rel_item(relevant=relevant, predicted=predicted)
    p_at_i = [0]*k
    k = min(len(relevant), k)
    for i in range(1, k+1):
        p_at_i[i-1] = precision_at_k(relevant=relevant, predicted=predicted, k=i)
    return sum([y*p/k for y, p in zip(y_i, p_at_i)])

def mnap_at_k(relevant, predicted, k: int = 10):
    """ 
        Функция расчета MAP@k
        relevant список по всем пользователям с их релевантными items
        predicted список по всем пользователям с их рекомендованными items
    """
    users = len(relevant)
    sum_napk = 0
    for user in range(users):
        sum_napk += nap_at_k(relevant=relevant[user], predicted=predicted[user], k=k)
    return sum_napk/users


def hitrate_at_k(relevant, predicted, k: int = 10):
    """
        Функция расчета Hitrate@k
        relevant список по всем пользователям с их релевантными items
        predicted список по всем пользователям с их рекомендованными items
    """
    
    cnt_user = len(predicted) # Количество пользователей    
    cnt_valid_user = 0
    for user in range(cnt_user):
        cnt_valid_user += int(len(set(relevant[user][:k]) & set(predicted[user][:k])) > 0) 

    return cnt_valid_user/cnt_user


def ndsg_at_k(relevant, predicted, k: int = 10):
    """
        Функция расчета nDSG@k
        relevant - релевантные items для одного пользователя
        predicted - рекомендованные items для одного пользователя
    """
    idsg_at_k = sum([1/np.log2(k+1) for k in range(1, k+1)])
    k = min(len(relevant), k)
    dsg_at_k = 0
    for k in range(1, k+1):
        dsg_at_k += int(relevant[k-1] == predicted[k-1])/np.log2(k+1)
    return dsg_at_k/idsg_at_k

In [31]:
def avg_precision_at_k(model, test_interactions, k:int=10, num_threads:int=60):
    p_at_k = precision_at_k(model=model, 
                            test_interactions=test_interactions,
                            k=10, 
                            num_threads=60)
    return sum(p_at_k)/len(p_at_k)

## Модель

In [32]:
class LFM():

    def __init__(self, encoder, rating_df=None, model=None):
        if model is None:
            self.model = LightFM(no_components=10,
                                loss='warp', 
                                random_state=42, 
                                learning_rate=0.01)
        else:
            self.model = model
        self.encoder = encoder
        self.rating_df = rating_df

    def predict(self, users_to_recommend: list, k: int = 10):
        """
            Параллельное вычисление рекомендаций для пользователей
        """
        predictions = {}
        for uid in users_to_recommend:
            predictions[uid] = self.recommend(uid=uid, k=k)
        return predictions

    def cold_start(self):
        """
            Функция холодного старта
            возвращает популярные продукты по убываюнию
        """
        if self.rating_df is None:
            return None
        return np.argsort(np.array(self.rating_df.groupby("product_id")["rating"].sum()))[::-1]


    def recommend(self,uid: int, k: int):
        """
            Расчет рекомендаций
        """
        # Если ранее такого пользователя не было, то применяется холодный старт
        try:
            uid = self.encoder.transform(np.array([uid]))[0]
        except:
            return self.cold_start()[:k]
        
        items = sorted(self.rating_df['product_id'].unique())
        scores = self.model.predict(user_ids=[uid]*len(items), 
                                    item_ids=items)
        predict = np.array(items)[np.argsort(-scores)][:k]
        return predict.tolist()
        
    def fit(self, train_sparse, epochs, rounds, num_threads=60):
        for rounds in tqdm(range(rounds)): 
            self.model.fit_partial(train_sparse, 
                            sample_weight=train_sparse, 
                            epochs=epochs, 
                            num_threads=num_threads)


In [33]:
# Инициализация
model = LightFM(no_components=10, loss='warp', 
                random_state=42, learning_rate=0.01)

lfm_model = LFM(encoder=encoder, rating_df=rating_df, model=model)

In [34]:
# Обучение
lfm_model.fit(train_sparse=train_sparse, epochs=5, 
              rounds=5, num_threads=60)

100%|██████████| 5/5 [00:28<00:00,  5.66s/it]


Оценки качества LightFM модели

In [58]:
# Пользователи с их релевантными покупками
relevant = (test_all_df.sort_values(by=["user_id", "add_to_cart_order"])
            .groupby(['user_id'])
            .agg({'product_id': 'unique'})['product_id'].to_list())

In [59]:
%%time
predict = lfm_model.predict(users_to_recommend=list(test_all_df['user_id'].unique()), k=10)
predict = list(predict.values())

In [ ]:
metrics = {}
metrics["MAP@k"] = map_at_k(relevant=relevant[:100], predicted=predict, k=10)
metrics["MNAP@k"] = mnap_at_k(relevant=relevant, predicted=predict, k=10)
metrics["Hitrate@k"] = hitrate_at_k(relevant=relevant, predicted=predict, k=10)
sum_ndsg = 0
sum_precision = 0
for user in range(len(relevant)):
    rel = relevant[user]
    pred = predict[user]
    sum_ndsg += ndsg_at_k(relevant=rel, predicted=pred, k=10)
    sum_precision += precision_at_k(relevant=rel, predicted=pred, k=10)
metrics["AVG_nDSG@k"] = sum_ndsg/len(relevant)
metrics["AVG_Precision@k"] = sum_precision/len(relevant)

In [ ]:
metrics

{'MAP@k': 0.007583333333333333,
 'MNAP@k': 0.013312962962962964,
 'Hitrate@k': 0.45,
 'AVG_nDSG@k': 0.01818597805998347,
 'AVG_Precision@k': 0.06699999999999996}

In [14]:
# Сохранение модели
with open("../app/models/lfm_model.pkl", "wb") as f:
    pickle.dump(lfm_model.model, f)

In [15]:
# Множество с рекоменлацией
predict_set = set(lfm_model.recommend(uid=3, k=10))
# Множество релевантных продуктов
relevant_set = set(np.array(init_test_df[init_test_df["user_id"] == 3]["product_id"]))
# Пересечение
relevant_set & predict_set 

{21903, 47766}

In [16]:
# Предикт для списка пользователей
lfm_model.predict(users_to_recommend=[1, 3, 2917], k=10)

{1: [13176, 6184, 16797, 21137, 196, 12341, 43352, 13575, 8571, 39275],
 3: [21137, 24852, 13176, 21903, 26209, 47209, 47626, 47766, 39275, 16797],
 2917: [21137, 24852, 21903, 13176, 47209, 26209, 47766, 47626, 22935, 24964]}